# December 16th - The floor will be lava...

In [31]:
example = [
    ".|...\\....",
    "|.-.\\.....",
    ".....|-...",
    "........|.",
    "..........",
    ".........\\",
    "..../.\\\\..",
    ".-.-/..|..",
    ".|....-|.\\",
    "..//.|...."
]

examplepartoneresult = 46

46

This one seems like the sort of problem that a is going to involve pathfinding on the second part of this.

In [32]:
struct Node
    nodetype::Char
    connections::Array{Union{Nothing,Node}}
    coordinates::Tuple{Int,Int}
end

function makenodes(contents)
    ## start by seeding the nodemap
    nodemap = Dict()
    for y in firstindex(contents, 1):lastindex(contents, 1)
        for x in firstindex(contents, 2):lastindex(contents, 2)
            if contents[y, x] != '.'
                nodemap[(y, x)] = Node(contents[y, x], fill(nothing, 4), (y, x))
            end
        end
    end
    ## connect the nodes
    for (_, node) in nodemap
        ## set outgoing connections
        (ny, nx) = node.coordinates
        ## north
        for y in range(ny - 1, firstindex(contents, 1), step=-1)
            if contents[y, nx] != '.' && contents[y, nx] != '|'
                node.connections[1] = nodemap[(y, nx)]
                break
            end
        end
        ## east
        for x in range(nx + 1, lastindex(contents, 1), step=1)
            if contents[ny, x] != '.' && contents[ny, x] != '-'
                node.connections[2] = nodemap[(ny, x)]
                break
            end
        end
        ## south
        for y in range(ny + 1, lastindex(contents, 1), step=1)
            if contents[y, nx] != '.' && contents[y, nx] != '|'
                node.connections[3] = nodemap[(y, nx)]
                break
            end
        end
        ## west
        for x in range(nx - 1, firstindex(contents, 1), step=-1)
            if contents[ny, x] != '.' && contents[ny, x] != '-'
                node.connections[4] = nodemap[(ny, x)]
                break
            end
        end
    end
    return nodemap
end

function createstartnode(nodemap, contents)
    startnode = Node('S', fill(nothing, 4), (1, 0))
    ## east
    for x in firstindex(contents, 1):lastindex(contents, 1)
        if contents[1, x] != '.' && contents[1, x] != '-'
            startnode.connections[2] = nodemap[(1, x)]
            break
        end
    end
    return startnode
end

function nextstep(node, direction, field)
    nextnode = node.connections[direction]
    if isnothing(nextnode)
        if direction == 1
            field[begin:node.coordinates[1], node.coordinates[2]] .= true
        elseif direction == 2
            field[node.coordinates[1], node.coordinates[2]:end] .= true
        elseif direction == 3
            field[node.coordinates[1]:end, node.coordinates[2]] .= true
        else
            field[node.coordinates[1], begin:node.coordinates[2]] .= true
        end
        return []
    end
    field[
        min(node.coordinates[1], nextnode.coordinates[1]):max(node.coordinates[1], nextnode.coordinates[1]),
        max(min(node.coordinates[2], nextnode.coordinates[2]), 1):max(node.coordinates[2], nextnode.coordinates[2])
    ] .= true
    if nextnode.nodetype == '|'
        return [(nextnode, 1), (nextnode, 3)]
    elseif nextnode.nodetype == '-'
        return [(nextnode, 2), (nextnode, 4)]
    elseif nextnode.nodetype == '/'
        return [(nextnode, if direction == 1
            2
        elseif direction == 2
            1
        elseif direction == 3
            4
        else
            3
        end)]
    else
        return [(nextnode, if direction == 1
            4
        elseif direction == 2
            3
        elseif direction == 3
            2
        else
            1
        end
        )]
    end
end

function stepall(node, direction, field)
    candidates = [(node, direction)]
    visited = Set()
    while length(candidates) > 0
        candidate = pop!(candidates)
        if candidate in visited
            continue
        end
        push!(visited, candidate)
        push!(candidates, nextstep(candidate[1], candidate[2], field)...)
    end
end

stepall (generic function with 1 method)

In [33]:
include("./aoc.jl")

function partone(contents)
    grid = makegrid(contents)
    field = fill(false, size(grid)...)
    nodes = makenodes(grid)
    startnode = createstartnode(nodes, grid)
    stepall(startnode, 2, field)
    return sum(field)
end

partone(example) == examplepartoneresult

true

## Part 2

Part 2 is all about find the most optimal start point. We can start on any node that doesn't have every edge connected.

In [34]:
function findstartnodes(nodemap, grid)
    starts = []
    for (_, node) in nodemap
        if isnothing(node.connections[1])
            push!(starts, (Node('S', [nothing, nothing, node, nothing], (firstindex(grid, 1), node.coordinates[2])), 3))
        end
        if isnothing(node.connections[2])
            push!(starts, (Node('S', [nothing, nothing, nothing, node], (node.coordinates[1], lastindex(grid, 2))), 4))
        end
        if isnothing(node.connections[3])
            push!(starts, (Node('S', [node, nothing, nothing, nothing], (lastindex(grid, 1), node.coordinates[2])), 1))
        end
        if isnothing(node.connections[4])
            push!(starts, (Node('S', [nothing, node, nothing, nothing], (node.coordinates[1], firstindex(grid, 2))), 2))
        end
    end
    return starts
end


function parttwo(contents)
    grid = makegrid(contents)
    nodes = makenodes(grid)
    startnodes = findstartnodes(nodes, grid)
    max(map(startnodes) do (startnode, direction)
        field = fill(false, size(grid)...)
        stepall(startnode, direction, field)
        sum(field)
    end...)
end

parttwo(example) == 51

true

## Results

In [35]:
execute(16, partone, parttwo)

7798
8026
